In [12]:
import os
import tifffile as tf
import numpy as np

base_dir = '/Users/julian/development/PycharmProjects/glioblastoma'

# calcium image
calcium_img_dir = os.path.join(base_dir, 'data', '01_raw', '276_AZD3965_Mathieu')
calcium_img_path = os.path.join(calcium_img_dir, '276_AZD3965_Mathieu.TIF')
directory = calcium_img_dir
image_path = calcium_img_path

# test stack
test_stack_dir = os.path.join(base_dir, 'data', '01_raw', 'high_res_test_stack')
test_stack_path = os.path.join(test_stack_dir, 'high_res_test_stack.tif')
directory = test_stack_dir
image_path = test_stack_path

# single image
# single_image_dir = '/Users/julian/development/PycharmProjects/glioblastoma/data/01_raw/single_image/'
# single_image_path = os.path.join(single_image_dir, 'single_image.tiff')



In [41]:
clusters = [[(1, 1), (1, 2), (1, 3), (2, 1)],
            [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (2, 0), (2, 2), (2, 3), (3, 0), (3, 1)], [(3, 2)], [(3, 3)]]
roi_clusters_dict = {(0, 0): 1, (0, 1): 1, (0, 2): 1, (0, 3): 1, (1, 0): 1, (1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 0): 1,
                     (2, 1): 0, (2, 2): 1, (2, 3): 1, (3, 0): 1, (3, 1): 1, (3, 2): 2, (3, 3): 3}

representative_signals = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                                   [0., 0., 0., 0., 4107.314, 10275.03, 15415.878, 10275.03, 0., 4107.314, 12936.118,
                                    15415.878, 0.],
                                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
representative_signals = representative_signals.astype(np.uint16)

n_clusters = 4
n_horizontal = 4
n_vertical = 4
roi_dims = (2048 // 4, 1536 // 4)
img_dims = (2048, 1536)

In [26]:
image = tf.imread(image_path)
n_frames, vertical_pixels, horizontal_pixels = image.shape

In [27]:
def create_signals_video():
    # for each cluster, we need to determine the pixels which it affects
    # Then for each frame, we need to set the pixel values of that frame to the signal value of the corresponding cluster
    # we do this for each cluster

    video_arr = np.zeros((n_frames, horizontal_pixels, vertical_pixels))


In [28]:
def create_frame(frame_idx: int):
    roi_width, roi_height = roi_dims

    frame_arr = np.zeros((horizontal_pixels, vertical_pixels))
    
    for roi, cluster in roi_clusters_dict.items():
        signal_value = representative_signals[cluster, frame_idx]

        # determine boundaries (they are exclusive, i.e. the upper value is not included)
        x_left, y_top = roi
        x_right, y_bottom = x_left + roi_width, y_top + roi_height

        frame_arr[x_left: x_right, y_top: y_bottom] = signal_value

    return frame_arr


In [37]:
video_arr = np.zeros((1, horizontal_pixels, vertical_pixels))

video_arr[0,:,:] = create_frame(1)
tf.imwrite('frame1.tif', video_arr, photometric=tf.PHOTOMETRIC.MINISBLACK)

In [ ]:
tf.imwrite('temp_C001T001.tif', np.random.rand(64, 64))

In [13]:
tif = tf.TiffFile(calcium_img_path)
tif.pages[0].tags['PhotometricInterpretation'].value


<PHOTOMETRIC.PALETTE: 3>

In [15]:
image = tf.imread(test_stack_path)

In [22]:
print(image.min(), ':', image.max())


0 : 255


In [30]:
image = tf.imread(calcium_img_path)

In [23]:
tf.imwrite('calcium_image.tif', image)

In [40]:
import numpy as np
import tifffile

# Create a sample NumPy array representing grayscale images
# Replace this with your actual numpy array
n_frames = 10
n_horizontal_pixels = 712
n_vertical_pixels = 512
data = np.random.randint(0, 2**16, size=(n_frames, n_horizontal_pixels, n_vertical_pixels), dtype=np.uint16)

# Define the output file name
output_file = 'output_multi_image.tif'

# Write the multi-image TIFF file
tf.imwrite(output_file, data.reshape(n_frames, n_vertical_pixels, n_horizontal_pixels))  # Adjust compress level as needed
